In [2]:
from __future__ import division
from pyspark import SparkConf, SparkContext
import re

* 设置应用的名称，设置需要去连接的url，最后是设置了一个配置属性，这里应该是key-value的形式

In [3]:
try:
    sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local").set("spark.cores.max", "4"))
except:
    pass

* 这里具体去实现的功能为一个分词的操作，后面应该是将文末的非字符换成""，前面是？？？

In [4]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

* 这里应该是一个数据集的生成

* 这里后面的map表示产生一个新的rdd,具体的生成方式是将这里的文本通过map中的func进行生成

In [5]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 4).map(parse_article)

In [6]:
result = wiki.take(1)[0]
for word in result[:50]:
    print word

Anarchism
Anarchism
is
often
defined
as
a
political
philosophy
which
holds
the
state
to
be
undesirable
unnecessary
or
harmful
The
following
sources
cite
anarchism
as
a
political
philosophy
Slevin
Carl
Anarchism
The
Concise
Oxford
Dictionary
of
Politics
Ed
Iain
McLean
and
Alistair
McMillan
Oxford
University
Press
2003
However
others
argue


In [7]:
def pairs_starting_from_word(words, first_word='word'):
    pairs = []
    
    for i, word in enumerate(words[:-1]):
        if (word == first_word):
            pair = '{}_{}'.format(word, words[i+1])
            cnt = 1
            pairs.append((pair, cnt))
        else:
            continue
    return pairs

In [27]:
# lowercase all words
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])

# find pairs starting from defined word
wiki_pairs = wiki_lower.flatMap(lambda x: pairs_starting_from_word(x, 'narodnaya'))

# filtering empty elements
wiki_pairs = wiki_pairs.filter(lambda x: x != [])

# aggregate counters
wiki_red = wiki_pairs.reduceByKey(lambda a, b: a + b, numPartitions=16)

# sort values by key
wiki_red_sorted = wiki_red.sortByKey()

In [9]:
result = wiki_red_sorted.collect()
for pair, cnt in result:
    print '{}\t{}'.format(pair, cnt)

narodnaya_gazeta	1
narodnaya_volya	9


In [17]:
# 获取了停词表
with open('/datasets/stop_words_en.txt', 'r') as f:
    stop_words = set(f.read().split())

print(len(stop_words))

319


In [18]:
def create_bigrams(words):
    bigrams = []
    for i, word in enumerate(words[:-1]):
        pair = u'_'.join((word, words[i+1])).encode('utf-8')
        cnt = 1
        bigrams.append((pair, cnt))
    return bigrams

In [19]:
from math import log
def calc_npmi(pair, cnt, words_occurrences_dict, total_num_of_words, total_num_of_pairs):
    word1, word2 = pair.split('_')
    p_a = words_occurrences_dict[word1] / total_num_of_words
    p_b = words_occurrences_dict[word2] / total_num_of_words
    
    pmi_ab = cnt / total_num_of_pairs
    pmi_a_b = log(pmi_ab / (p_a * p_b))
    
    nmpi_a_b = pmi_a_b / -log(pmi_ab)
    return (pair, nmpi_a_b)

In [20]:
# lowercase all words
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])

# words not in stop_words_en.txt
wiki_filt = wiki_lower.map(lambda words: [x for x in words if x not in stop_words])

# create bigrams
wiki_bigrams = wiki_filt.flatMap(create_bigrams)

# aggregate counters
wiki_red = wiki_bigrams.reduceByKey(lambda a, b: a + b)

# filter values by counter
wiki_red_filt = wiki_red.filter(lambda (pair, cnt): cnt >= 500)

In [21]:
# total number of words
tot_num_words = wiki_filt.map(lambda words: len(words))
tot_num_words = tot_num_words.reduce(lambda a, b: a + b)
tot_num_words

6971026

In [22]:
# total number of words pairs
tot_num_pairs = wiki_filt.map(lambda words: len(words) - 1)
tot_num_pairs = tot_num_pairs.reduce(lambda a, b: a + b)
tot_num_pairs

6966926

In [23]:
# number of each word occurrences
words_occ = wiki_filt.flatMap(lambda words: [(x, 1) for x in words])
words_occ = words_occ.reduceByKey(lambda a, b: a + b)
words_occ = words_occ.filter(lambda (pair, cnt): cnt >= 500)
words_occ = words_occ.collect()

words_occ_dict = dict()
for item, cnt in words_occ:
    words_occ_dict[item] = cnt

In [24]:
pairs_npmi = wiki_red_filt\
    .map(lambda (pair, cnt): calc_npmi(pair, cnt, words_occ_dict, tot_num_words, tot_num_pairs))\
    .map(lambda (a, b): (b, a))\
    .sortByKey(False)\
    .map(lambda (a, b): (b, a))\
    .take(39)

In [25]:
print(len(pairs_npmi))

39


In [26]:
for pair, npmi in pairs_npmi:
    print pair

los_angeles
external_links
united_states
prime_minister
san_francisco
et_al
new_york
supreme_court
19th_century
20th_century
references_external
soviet_union
air_force
baseball_player
university_press
roman_catholic
united_kingdom
references_reading
notes_references
award_best
north_america
new_zealand
civil_war
catholic_church
world_war
war_ii
south_africa
took_place
roman_empire
united_nations
american_singer-songwriter
high_school
american_actor
american_actress
american_baseball
york_city
american_football
years_later
north_american
